# Выбор локации для скважины

# Описание проекта
Нужно решить, где бурить новую скважину. 
Шаги для выбора локации обычно такие:
- В избранном регионе собирают характеристики для скважин: качество нефти и объём её запасов;
- Строят модель для предсказания объёма запасов в новых скважинах;
- Выбирают скважины с самыми высокими оценками значений;
- Определяют регион с максимальной суммарной прибылью отобранных скважин.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. 
- Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. 
- Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# Инструкция по выполнению проекта

Загрузите и подготовьте данные. Поясните порядок действий.

Обучите и проверьте модель для каждого региона:

- 2.1. Разбейте данные на обучающую и валидационную выборки в соотношении 75:25.
- 2.2. Обучите модель и сделайте предсказания на валидационной выборке.
- 2.3. Сохраните предсказания и правильные ответы на валидационной выборке.
- 2.4. Напечатайте на экране средний запас предсказанного сырья и RMSE модели.
- 2.5. Проанализируйте результаты.

Подготовьтесь к расчёту прибыли:

- 3.1. Все ключевые значения для расчётов сохраните в отдельных переменных.
- 3.2. Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе. 
- 3.3. Напишите выводы по этапу подготовки расчёта прибыли.

Напишите функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:
- 4.1. Выберите скважины с максимальными значениями предсказаний.
- 4.2. Просуммируйте целевое значение объёма сырья, соответствующее этим предсказаниям.
- 4.3. Рассчитайте прибыль для полученного объёма сырья.

Посчитайте риски и прибыль для каждого региона:
- 5.1. Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
- 5.2. Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
- 5.3. Напишите выводы: предложите регион для разработки скважин и обоснуйте выбор.

# Описание данных
Данные геологоразведки трёх регионов находятся в файлах: 
- /datasets/geo_data_0.csv
- /datasets/geo_data_1.csv
- /datasets/geo_data_2.csv


- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

# Условия задачи
- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.
- Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.`

## Загрузка и подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from scipy import stats as st
import numpy as np

In [2]:
data0 = pd.read_csv('/datasets/geo_data_0.csv')
data0.info()
data0.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [3]:
data1 = pd.read_csv('/datasets/geo_data_1.csv')
data1.info()
data1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [4]:
data2 = pd.read_csv('/datasets/geo_data_2.csv')
data2.info()
data2.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


# Зададим целевые признаки и разобьём данные на выборки

In [5]:
# зададим целевые признаки 
target0 = data0['product']
features0 = data0.drop(['product', 'id'], axis=1)
# разобьём данные на выборки
features_train0, features_valid0, target_train0, target_valid0 = train_test_split(
    features0, target0, test_size=0.25, random_state=12345)

In [6]:
target1 = data1['product']
features1 = data1.drop(['product', 'id'], axis=1)
features_train1, features_valid1, target_train1, target_valid1 = train_test_split(
    features1, target1, test_size=0.25, random_state=12345)

In [7]:
target2 = data2['product']
features2 = data2.drop(['product', 'id'], axis=1)
features_train2, features_valid2, target_train2, target_valid2 = train_test_split(
    features2, target2, test_size=0.25, random_state=12345)

## Обучение и проверка модели

In [8]:
model0 = LinearRegression()
model0.fit(features_train0, target_train0) 
predictions_valid0 = model0.predict(features_valid0)
result0 = mean_squared_error(target_valid0, predictions_valid0, squared = False) 
print("RMSE:", result0)
print("Среднее:", predictions_valid0.mean())

RMSE: 37.5794217150813
Среднее: 92.59256778438035


In [9]:
model1 = LinearRegression()
model1.fit(features_train1, target_train1)
predictions_valid1 = model1.predict(features_valid1)
result1 = mean_squared_error(target_valid1, predictions_valid1, squared = False) 
print("RMSE:", result1)
print("Среднее:", predictions_valid1.mean())

RMSE: 0.893099286775617
Среднее: 68.728546895446


In [10]:
model2 = LinearRegression()
model2.fit(features_train2, target_train2)
predictions_valid2 = model2.predict(features_valid2) 
result2 = mean_squared_error(target_valid2, predictions_valid2, squared = False)
print("RMSE:", result2)
print("Среднее:", predictions_valid2.mean())

RMSE: 40.02970873393434
Среднее: 94.96504596800489


# Выводы:
- наименьшими средними предсказанными запасами обладает 2 регион, при наименьшей ошибке
- наибольшими средними предсказанными запасами обладает 3 регион, при наибольшей ошибке

## Подготовка к расчёту прибыли

In [11]:
# Переменные
budget = 10000000000
product_revenue = 450000
barrel_revenue = 450
wells_total = 500
wells_selected = 200
risk_level = 0.025

In [12]:
one_well_budget = budget/wells_selected

one_well_product_need = one_well_budget/product_revenue
print('Минимальный необходимый уровень запасов в скважине для разработки:',round(one_well_product_need,1))

mean_oil_reserves=[predictions_valid0.mean(),predictions_valid1.mean(),predictions_valid2.mean()]
print()
print('Оценка рентабельности регионов по средним запасам')
print()
def reserves(mean_reserves):
    for i in range(len(mean_oil_reserves)):
        if mean_reserves[i] < one_well_product_need:
            print(f'Запас сырья в регионе {i+1} меньше необходимого на {round(one_well_product_need - mean_oil_reserves[i], 1)}')
        else:
            print(f'Запас сырья в регионе {i+1} больше необходимого на {round(mean_oil_reserves[i] - one_well_product_need, 1)}')

reserves(mean_oil_reserves)

Минимальный необходимый уровень запасов в скважине для разработки: 111.1

Оценка рентабельности регионов по средним запасам

Запас сырья в регионе 1 меньше необходимого на 18.5
Запас сырья в регионе 2 меньше необходимого на 42.4
Запас сырья в регионе 3 меньше необходимого на 16.1


# Функция для расчёта прибыли по выбранным скважинам и предсказаниям модели:
- 4.1. Выберите скважины с максимальными значениями предсказаний. 
- 4.2. Просуммируйте целевое значение объёма сырья, соответствующее этим предсказаниям.
- 4.3. Рассчитайте прибыль для полученного объёма сырья.

In [13]:
def profit(target, predictions):
    target = pd.Series(target).reset_index(drop=True)
    predictions = pd.Series(predictions).reset_index(drop=True)
    predictions_selection = predictions.sort_values(ascending=False)[:wells_selected].index
    selection = target[predictions_selection]
    profit = (selection.sum() * product_revenue) - budget
    return round(profit, 1)

In [14]:
print('Прибыль в регионе 0:',profit(target_valid0, predictions_valid0))

Прибыль в регионе 0: 3320826043.1


In [15]:
print('Прибыль в регионе 1:',profit(target_valid1, predictions_valid1))

Прибыль в регионе 1: 2415086696.7


In [16]:
print('Прибыль в регионе 2:',profit(target_valid2, predictions_valid2))

Прибыль в регионе 2: 2710349963.6


## Расчёт прибыли и рисков 

# Применим Bootstrap для выбора региона

In [17]:
def bootstrap(target, probabilities, count, sample_size):
    state = np.random.RandomState(12345)
    values = []
    for i in range(count):
        target_subsample = target.sample(n=sample_size, random_state=state, replace=True)
        probs_subsample = probabilities[target_subsample.index]
        
        values.append(profit(target_subsample, probs_subsample))

    values = pd.Series(values)
    mean = values.mean()
    risk = (values < 0).mean()
    
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    quantile_interval = (lower, upper)
    return round(mean,1), round(risk*100, 1), quantile_interval

# Регион 0

In [18]:
mean0, risk0, interval0= bootstrap(target_valid0.reset_index(drop=True), predictions_valid0, 1000, 500)
print('Средняя выручка = ', mean0)
print('95 %ый доверительный интервал по квантилям = ', interval0)
print(f'Риск убытков: {risk0}%')

Средняя выручка =  396164984.8
95 %ый доверительный интервал по квантилям =  (-111215545.89999999, 909766941.5049999)
Риск убытков: 6.9%


# Регион 1

In [19]:
mean1, risk1, interval1, = bootstrap(target_valid1.reset_index(drop=True), predictions_valid1, 1000, 500)
print('Средняя выручка = ', mean1)
print('95 %ый доверительный интервал по квантилям = ', interval1)
print(f'Риск убытков: {risk1}%')

Средняя выручка =  456045105.8
95 %ый доверительный интервал по квантилям =  (33820509.38750001, 852289453.9024999)
Риск убытков: 1.5%


# Регион 2

In [20]:
mean2, risk2, interval2, = bootstrap(target_valid2.reset_index(drop=True), predictions_valid2, 1000, 500)
print('Средняя выручка = ', mean2)
print('95 %ый доверительный интервал по квантилям = ', interval2)
print(f'Риск убытков: {risk2}%')

Средняя выручка =  404403866.6
95 %ый доверительный интервал по квантилям =  (-163350413.355, 950359574.885)
Риск убытков: 7.6%


# Вывод:
- по результатам исследования, предпочтителен для разработки регион 1. в пользу этого говорят следующие параметры:
- наибольшая средняя выручка
- лучшие значения в пределах 95% доверительного интервала
- наименьшие риски. к тому же этот регион единственный, в котором уровень рисков удовлетворяет условию задачи

## Чек-лист готовности проекта

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован